In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementations import *
from utilitaries import *
from functions import *



In [2]:
from proj1_helpers import *

labels, raw_data, indices = load_csv_data('train.csv', sub_sample=False)
nan_data = meaningless_to_nan(raw_data)


In [3]:
n_samples, n_features = np.shape(nan_data)
features = []
for feature in range(n_features):
    for sample in range(n_samples):
        if np.isnan(nan_data[sample,feature]):
            features.append(feature)
nan_columns = np.unique(features)
print(nan_columns)   

[ 0  4  5  6 12 23 24 25 26 27 28]


In [4]:
# estimation of columns
new_submatrix = np.delete(nan_data,nan_columns, axis = 1)
k_fold = 4
seed = 2
lambdas = np.logspace(-5,-1,5)
#accuracy = np.ones(len(lambdas))
loss_te = np.ones(len(lambdas))
loss_tr = np.ones(len(lambdas))
for chosen_feature in nan_columns:
    samples = []
    for sample in range(n_samples):
        if np.isnan(nan_data[sample,chosen_feature]):
            samples.append(sample)
    nan_lines = samples
    new_submatrix0 = np.delete(new_submatrix,nan_lines, axis = 0)
    labels_0 = np.delete(nan_data[:,chosen_feature],nan_lines, axis = 0)
    k_indices = build_k_indices(labels_0, k_fold, seed)
    for idx, lambda_ in enumerate(lambdas):
        loss_tr[idx],loss_te[idx] = cross_validation(labels_0, new_submatrix0, k_indices, k_fold, lambda_, degree=1)
    best_lambda = lambdas[np.argmin(loss_te)]
    print((loss_te))
    weights, loss = ridge_regression(labels_0, new_submatrix0, best_lambda)
    x_pred = np.dot(new_submatrix[nan_lines,:], weights)
    nan_data[nan_lines,chosen_feature] = x_pred
    #print(x_pred.shape)
estimated_data = nan_data

[162.69645409 162.6964708  162.69679268 162.71342803 163.63473225]
[1.51285956 1.51285928 1.5128605  1.51318391 1.52719464]
[71292.7858678  71292.76090572 71292.78801144 71304.62961317
 71626.52338919]
[6.1883634  6.18836306 6.18836519 6.1888764  6.20272531]
[0.07897503 0.07897501 0.07897509 0.07898531 0.07942197]
[180.47708921 180.47714094 180.48201155 180.90203923 195.7784728 ]
[1.48914632 1.48914619 1.48914561 1.48914758 1.48978557]
[1.34909665 1.34909649 1.34909526 1.34909244 1.34984758]
[170.39793845 170.39787504 170.39759551 170.41733194 171.08817577]
[1.99853974 1.9985395  1.99853853 1.99853454 1.99892813]
[1.58920892 1.58920895 1.58920826 1.58920449 1.58935043]


In [5]:
degrees = range(5)
lambdas = np.logspace(-5, -1, 10)
k_fold = 4
seed = 27
k_idx = build_k_indices(labels, k_fold, seed)
#loss_tr = np.ones((len(degrees),len(lambdas)))
loss_te = np.ones((len(degrees),len(lambdas)))
scores = np.ones((len(degrees),len(lambdas)))
#labels = np.expand_dims(labels, axis=1) 

for degree_idx, degree in enumerate(degrees):
    for lambda_idx, lambda_ in enumerate(lambdas):
        _ ,loss_te[degree_idx, lambda_idx], scores[degree_idx, lambda_idx] = cross_validation_(labels, estimated_data, k_idx, k_fold, lambda_, degree)
        print(degree_idx,lambda_idx)
best_HP_idx = np.unravel_index(np.argmin(loss_te),np.shape(loss_te))
best_degree = degrees[best_HP_idx[0]]
best_lambda = lambdas[best_HP_idx[1]]
print(best_degree, best_lambda)

#weights, loss = ridge_regression(labels,estimated_data, best_lambda)


1 1e-05
